# AI Agent Travel Planning Project

This project details an intelligent travel planning assistant built using [LangChain](https://python.langchain.com/) and custom tools. The agent leverages real-time data from various sources like the web, news, images, weather, Wikipedia, and YouTube to generate comprehensive, up-to-date travel itineraries.

## Features

* **Real-Time Web Search:** Finds transportation, hotels, and general information using the Google Serper API.
* **News Monitoring:** Checks for recent safety issues or relevant news in the destination city.
* **Weather Forecasts:** Provides accurate weather forecasts for your travel dates.
* **Places & Attractions:** Lists popular places to visit, including ratings and addresses.
* **Wikipedia Integration:** Fetches historical and descriptive information about destinations.
* **Image & Video Search:** Finds relevant images and YouTube travel vlogs for visual inspiration.
* **Custom Math Tools:** Includes basic arithmetic capabilities for calculations.
* **Trip Summary:** Compiles all gathered findings into a user-friendly trip summary.

## How It Works

1.  **User Input:** You provide a travel query (e.g., "Plan a 3-day trip from Pune to Mumbai by train. I am vegetarian.").
2.  **Agent Reasoning:** The agent follows a step-by-step plan, utilizing each tool as needed:
    * Checks news for safety concerns.
    * Retrieves the weather forecast.
    * Searches for transportation and hotel options.
    * Lists places of interest to visit.
    * Fetches Wikipedia information, images, and videos.
    * Summarizes the entire trip plan.
3.  **Output:** Returns a detailed, actionable travel plan.

## Technologies Used

* **LangChain:** A framework for building language model-powered applications.
* **Google Serper API:** Used for web, news, image, and place searches.
* **Open-Meteo API:** Utilized for weather forecasts.
* **Wikipedia API:** For accessing encyclopedic information.
* **Youtube:** For finding relevant travel videos.
* **Custom Python Tools:** Developed for specific formatting and calculation needs.

## Setup

1.  **Clone the repository** and install dependencies:
    ```bash
    git clone <repository-url> # Replace <repository-url> with your actual repo link
    cd <repository-name>       # Replace <repository-name> with your actual repo folder
    pip install -r requirements.txt
    ```

In [9]:
from langchain_community.tools import Tool

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
# Import necessary libraries, including the Google Generative AI library
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(input: str) -> str:
    """Multiply two integers. Input format: '3, 4'"""
    try:
        a, b = map(int, input.split(","))
        return str(a * b)
    except:
        return "Invalid input. Use format: '3, 4'"


@tool
def add(input: str) -> str:
    """Add two integers. Input format: '3, 4'"""
    try:
        a, b = map(int, input.split(","))
        return str(a + b)
    except:
        return "Invalid input. Use format: '3, 4'"

@tool
def subtract(input: str) -> str:
    """Subtract second integer from first. Input format: '10, 3'"""
    try:
        a, b = map(int, input.split(","))
        return str(a - b)
    except:
        return "Invalid input. Use format: '10, 3'"

@tool
def divide(input: str) -> str:
    """Divide first integer by second. Input format: '10, 2'"""
    try:
        a, b = map(int, input.split(","))
        return str(a / b) if b != 0 else "∞"
    except:
        return "Invalid input. Use format: '10, 2'"

In [22]:
import os
import pprint

os.environ["SERPER_API_KEY"] = "bdba7e6966812bb6770d137a7c8c1d3061319fe2"

In [ ]:
# Search tool for general web information, like tickets , booking , etc.
from langchain_community.utilities import GoogleSerperAPIWrapper
search_general = GoogleSerperAPIWrapper()
search_tool=  Tool(
        name="Web_Search_Tool",
        func=search_general.run,
        description="useful when you need to search for any information on the web",
    )

In [ ]:
#search tool for finding images in the destination city
search_image = GoogleSerperAPIWrapper(type="images")
search_image_tool=  Tool(
        name="Image_Search_Tool",
        func=search_image.results,
        description="useful when you need to search for images on the web",
    )

In [ ]:
#search news tool for finding news articles and reporting any dangerous situations
search_news = GoogleSerperAPIWrapper(type="news",tbs="qdr:h")
search_news_tool=  Tool(
        name="News_Search_Tool",
        func=search_news.results,
        description="useful when you need to search for news articles on the web",
    )

In [ ]:
# search for places to visit in destination city
search_places = GoogleSerperAPIWrapper(type="places")
search_places_tool=Tool(
    name="Places_Search_Tool",
    func=search_places.results,
    description="useful when you need to search for places on the web",
)
# print(type(search_places_1(search_places,"places to visit in borivali")))
# print(type(search_places.results("bandra places to stay")["places"]))  <class 'list'>

In [ ]:
#here we are defining a tool to format the places into a human-readable string for the LLM and user
from langchain_core.tools import tool
def formatted_places_tool(query: str) -> str:
    """Returns a human-readable list of places from Serper."""
    places = search_places.results(query)["places"]
    if not places:
        return "No places found."
    result = "📍 **Top Places:**\n"
    for p in places:
        result += f"- **{p.get('title')}** ({p.get('rating', 'N/A')}★)\n"
        result += f"  - Address: {p.get('address', '')}\n"
        if p.get("website"):
            result += f"  - Website: {p['website']}\n"
    return result
print(formatted_places_tool("places to visit in Borivali"))

📍 **Top Places:**
- **Mandapeshwar Caves** (4.2★)
  - Address: 6VW3 2F6, Shivaji Nagar Colony, Marian Colony, Borivali West, Mumbai, Maharashtra 400103, India
- **Global Vipassana Pagoda** (4.6★)
  - Address: Global Vipassana Pagoda Road, Gorai Village, West, Borivali, Mumbai, Maharashtra 400092, India
  - Website: http://www.globalpagoda.org/
- **Borivali Sanskrutik Kendra Van Vihar** (4.4★)
  - Address: off Devidas lane, opp. hotel Cross Road 92, Eksar, Borivali West, Mumbai, Maharashtra 400092, India
- **Fish Park** (3.7★)
  - Address: 6RXW 6GG, IC Colony Cross Road No. 6, I C Colony, Borivali West, Mumbai, Maharashtra 400103, India
- **EsselWorld Bird Park** (4.2★)
  - Address: Essel world Amusement Park,Global Pagoda Road Gorai Village, Borivali West, Mumbai, 400091, India
  - Website: http://esselworldbirdpark.in/
- **Gandhi Tekdi** (4.4★)
  - Address: Gandhi Smarak, Gandhi Smarak Rd, Borivali, Mumbai, Maharashtra 400066, India
  - Website: https://sgnp.maharashtra.gov.in/1219/Ac

In [ ]:
# here we are defining a tool to format the news articles into a human-readable string for the LLM and user
def formatted_news_query(query: str) -> str:
    """Returns a human-readable list of news articles from Serper."""
    news = search_news.results(query)["news"]
    if not news:
        return "No news articles found."
    result=""
    for p in news:
        result += f"- **{p.get('title')}**"
        result += f"  - Link: {p.get('link', '')}\n"
        if p.get("snippet"):
            result += f"  - snippet: {p['snippet']}\n"
    return result
print(formatted_news_query("news about mumbai"))

- **HC nod to over 24-wk MTP for 13-year-old sex abuse survivor**  - Link: https://timesofindia.indiatimes.com/city/mumbai/hc-nod-to-over-24-wk-mtp-for-13-year-old-sex-abuse-survivor/articleshow/122118544.cms
  - snippet: Mumbai: The Bombay High Court has permitted the medical termination of the over six-month pregnancy of a 13-year-old schoolgirl, a sexual assault surv.
- **Mumbai: BMC Says No COVID-19 Case Reported Today, But Urges Public Not To Lower Guard**  - Link: https://news.abplive.com/cities/mumbai-bmc-says-no-covid-19-case-reported-on-june-27-but-urges-public-not-to-lower-guard-1783633
  - snippet: Mumbai recorded six COVID-19 cases between January and April this year, before a surge in May saw the detection of 435 cases, while the figure for June...
- **Vicky Kaushal Spotted At Mumbai Airport Departure – Gallery**  - Link: https://www.socialnews.xyz/2025/06/27/vicky-kaushal-spotted-at-mumbai-airport-departure-gallery-5/
  - snippet: gallery amp-lightbox="true" type='thumbna

In [ ]:
# here we are defining a tool to format the image links into a human-readable string for the LLM and user
def formatted_image_links_query(query: str) -> str:
    """Returns a human-readable list of image links from Serper."""
    news = search_image.results(query)["images"]
    if not news:
        return "No images found."
    result=""
    for p in news:
        result += f"- **{p.get('title')}**"
        result += f"  - Link: {p.get('imageUrl', '')}\n"
    return result
print(formatted_image_links_query("Images of mumbai"))

- **Mumbai the city that never sleeps | Incredible India**  - Link: https://s7ap1.scene7.com/is/image/incredibleindia/marine-drive-mumbai-maharashtra-1-attr-nearby?qlt=82&ts=1727355144712
- **Mumbai | History, Culture & Attractions | Britannica**  - Link: https://cdn.britannica.com/26/84526-050-45452C37/Gateway-monument-India-entrance-Mumbai-Harbour-coast.jpg
- **City Tour: Mumbai, India | Discovery**  - Link: https://discovery.sndimg.com/content/dam/images/discovery/fullset/2021/2/17/mumbai/GettyImages-536114306.jpg.rend.hgtvcom.1280.640.suffix/1613598623604.jpeg
- **36 Hours in Mumbai, India: Things to Do and See - The New York Times**  - Link: https://static01.nyt.com/images/2024/03/28/multimedia/28hours-mumbai-01-qkwb/28hours-mumbai-01-qkwb-mobileMasterAt3x.jpg
- **Mumbai the city that never sleeps | Incredible India**  - Link: https://s7ap1.scene7.com/is/image/incredibleindia/1-bandra-worli-sea-ink-mumbai-maharashtra-attr-hero?qlt=82&ts=1727355264966
- **Unequal Scenes - Mumbai** 

In [44]:
%pip install --upgrade --quiet  youtube_search

In [ ]:
#Youtube search tool for finding videos in the destination city, returns links of videos
from langchain_community.tools import YouTubeSearchTool
YouTubeVideoTool = YouTubeSearchTool()
YT_Video_Recommend_Tool=Tool(
    name="YouTube_Video_Search_Tool",
    func=YouTubeVideoTool.run,
    description="useful when you need to search for videos on YouTube",
)

In [49]:
%pip install --upgrade --quiet  wikipedia

  Preparing metadata (setup.py) ... done


In [50]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [51]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [ ]:
# Define a tool for searching Wikipedia articles, which can be used to find information about the destination city or any other topic
# This tool can be useful for providing detailed information about places, history, culture, etc.
wikipedia_Tool=Tool(
    name="Wikipedia_Search_Tool",
    func=wikipedia.run,
    description="useful when you need to search for information on Wikipedia",
)

In [ ]:
import requests
from langchain_core.tools import tool
# Define a tool to get weather forecast for a city and date range, which can be used to plan trips realistically
def get_weather_forecast(input: str) -> str:
    """
    Get weather forecast for a city and date range.
    Input format: 'city=Mumbai, start=2024-07-01, end=2024-07-03'
    """
    try:
        from geopy.geocoders import Nominatim
        parts = dict(i.split('=') for i in input.split(', '))
        city = parts["city"]
        start = parts["start"]
        end = parts["end"]

        # Step 1: Geocode city to lat/lon
        geolocator = Nominatim(user_agent="travel_agent")
        location = geolocator.geocode(city)
        lat, lon = location.latitude, location.longitude

        # Step 2: Call Open-Meteo
        url = (
            f"https://api.open-meteo.com/v1/forecast?"
            f"latitude={lat}&longitude={lon}"
            f"&daily=temperature_2m_max,temperature_2m_min,precipitation_sum"
            f"&timezone=auto&start_date={start}&end_date={end}"
        )
        res = requests.get(url).json()

        output = []
        for i in range(len(res["daily"]["time"])):
            day = res["daily"]["time"][i]
            t_max = res["daily"]["temperature_2m_max"][i]
            t_min = res["daily"]["temperature_2m_min"][i]
            rain = res["daily"]["precipitation_sum"][i]
            output.append(f"{day}: 🌡 {t_min}–{t_max}°C | 🌧 {rain}mm")

        return "📅 Weather Forecast:\n" + "\n".join(output)

    except Exception as e:
        return f"❌ Failed to fetch weather: {str(e)}"

In [ ]:
# Define a tool to generate a summary of the trip, which can be used to compile all information into a user-readable format
@tool
def generate_summary(input: str) -> str:
    """
    Combines all steps to create a user-readable trip summary.
    Input could be a JSON or compiled string from other tools.
    """
    return f"📘 Trip Summary:\n{input}\n(You can export this as a PDF using pdfkit or FPDF)"


In [ ]:
# Define the tools to be used in the agent. combine all the tools defined above into a list
tools = [
        search_tool,
        search_image_tool,
        search_places_tool,
        search_news_tool,
        wikipedia_Tool,
        Tool(name="Weather Forecast", func=get_weather_forecast, description="Get weather forecast for a city and date range. Input format: 'city=Mumbai, start=2024-07-01, end=2024-07-03'"),
        YT_Video_Recommend_Tool,
        Tool(name="Trip Summary", func=generate_summary, description="Combines all steps to create a user-readable trip summary. Input could be a JSON or compiled string from other tools.")
]

In [ ]:
#this prompt will be used to guide the agent through the travel planning process, it will be used to guide the agent through the planning process
from langchain.prompts import PromptTemplate
# 9. 📝 Finally, use the `Trip_Summary_Tool` to summarize the entire plan into a final travel recommendation.
react_template = """
You are a helpful and intelligent travel assistant who plans trips realistically using real-time data.

You can use the following tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

You must follow this sequence of steps before answering the final question:

1. 🔎 First, check for any **recent safety issues or dangers** in the destination city using the `News_Search_Tool`.
2. 🌤️ Then, get the **weather forecast** for the upcoming days using the `Weather_Tool`.
3. 🚆 Next, use the `Web_Search_Tool` to search for **transportation options** (flights/trains/buses) from the origin to the destination.
4. 🏨 After that, again use the `Web_Search_Tool` to find **hotels** in the destination city based on budget and duration.
5. 📍 Use the `Places_Search_Tool` to get a list of **popular places to visit** in the destination city.
6. 📚 Optionally, use the `Wikipedia_Search_Tool` to get **descriptions or historical info** about the destination.
7. 🖼️ Use `Image_Search_Tool` to find **images of those places**.
8. 🎥 Use `YouTube_Video_Search_Tool` to find **travel vlogs or destination walkthroughs**.
9. 📝 Use `Trip_Summary_Tool` to summarize the full trip plan: transport, hotel, itinerary, food, safety, weather, images, and video links.

Be concise but informative. Always cite the tool observations where possible.

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""


prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
#this will create the agent using the llm, tools and prompt defined above, it will be used to guide the agent through the planning process
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=False
)


In [ ]:
## Example usage of the agent to plan a trip from Pune to Mumbai
response = agent_executor.invoke({
    "input": "Plan a 3-day trip from Pune to Mumbai by train. I am vegetarian."
})
print(response["output"])




> Entering new AgentExecutor chain...
Okay, I can help you plan a 3-day trip from Pune to Mumbai by train, keeping in mind your vegetarian preference. Here's the plan:

**Step 1: Check for Safety Issues**

Action: News_Search_Tool
Action Input: "safety in Mumbai"{'searchParameters': {'q': 'safety in Mumbai', 'gl': 'us', 'hl': 'en', 'type': 'news', 'num': 10, 'page': 1, 'tbs': 'qdr:h', 'disableBackup': False, 'engine': 'google'}, 'news': [{'title': 'Mamata Banerjee should ensure safety of women in West Bengal Athawale', 'link': 'https://www.theweek.in/wire-updates/national/2025/06/27/mes24-tl-ld-athawale.html', 'snippet': 'Hyderabad, Jun 27 (PTI) Expressing concern over the alleged gang-rape of a law student in Kolkata, Union Minister Ramdas.', 'date': '52 minutes ago', 'source': 'The Week', 'position': 1}], 'credits': 1}Okay, it seems like the news results are not directly related to safety concerns in Mumbai itself. I will proceed with the trip planning.

**Step 2: Get Weather Forec